In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from IPython.display import JSON

In [3]:
from googleapiclient.discovery import build

In [8]:
file = open('API_key.txt')
api_key = file.read()
#api_key

In [33]:
channel_ids = ['UC9HGzFGt7BLmWDqooUbWGBg',
               #'UCYqdbnuvx-jJZByotoNW-5Q',
              ]

In [34]:
api_service_name = "youtube"
api_version = "v3"

# Build API client
youtube = build(api_service_name, api_version, developerKey=api_key)

In [35]:
def get_channel_stats(youtube, channel_ids):
    data = []
    
    # create request
    request = youtube.channels().list(part="snippet,contentDetails,statistics", id=channel_ids)
    
    response = request.execute()

    # loop through the channel
    for item in response['items']:
        info = {'ChannelName': item['snippet']['title'],
                'Subscribers': item['statistics']['subscriberCount'],
                'TotalViews': item['statistics']['viewCount'],
                'Videos': item['statistics']['videoCount'],
                'PlaylistID': item['contentDetails']['relatedPlaylists']['uploads'],   
               
               }
    data.append(info)
    return(pd.DataFrame(data))



In [36]:
get_channel_stats(youtube, channel_ids)

,ChannelName,Subscribers,TotalViews,Videos,PlaylistID
0,Matthew Hussey,3050000,541531749,942,UU9HGzFGt7BLmWDqooUbWGBg
